# 4章 データ前処理 -よりよいデータセットの構築-

2018/05/18

- データセットにおける欠測値の削除と補完
- 機械学習アルゴリズムに合わせたカテゴリデータの整形
- モデルの構築に適した特徴量の選択

## 4.1 欠測データへの対処


### 4.1.1 欠測値を含む要素を取り除く

- 一般的な計算ツールは欠測値へのできない
- 無視すると予期せぬ結果を生み出す
- 分析者が適切に対応することが重要

- pandas.DataFrame.dropna が便利。
    - デメリット: 削除しすぎると解析の信頼性が失われる可能性もある。
        - --> 補完（次項）の方法
        
### 4.1.2 補完

- 平均補完
    - サンプルの平均値で補完
    - sklearnのImputerにはstrategyにmedian, やmost_frequentなどもある


## 4.2 カテゴリデータの対処

- 順序尺度のデータを自動的に整数値に置き換えることは困難
    - 自前でマッピングを定義して生成する必要がある。

- 順序の性質を持たないクラスラベルには0から値を設定するなど
- ↑と同様もしくは、sklean LabelEncoder

- one-hot エンコーディング
    - 整数値にしてしまうと、順序の意味を持ってしまう。
        - 学習アルゴリズムによっては順序の意味を解釈してしまうかもしれない。
        - 問題を回避する方法としてone-hotエンコーディングという手法がある

## 4.3 データセットをトレーニングデータとテストデータと分割する

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [5]:
import pandas as pd

df = pd.DataFrame([
    ['green', 'M', 10.1, 'class1'], 
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1']
])

# 列名を設定
df.columns = ['color', 'size', 'price' , 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [32]:
# 順序尺度のデータを整数値に自動的に置き換えることはできないので、自分で定義する必要がある

# Tシャツのサイズと整数値をマッピング

size_mapping = {
    'XL': 3,
    'L': 2,
    'M': 1
}

_df = df.copy()
_df['size'] = _df['size'].map(size_mapping)
_df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [33]:
# マッピングを元に

inv_size_mapping = { v:k for k,v in size_mapping.items()}
__df = _df.copy()
__df.size = __df['size'].map(inv_size_mapping)
__df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [34]:
import numpy as np


# クラスラベルを整数値に対応させるディクショナリを生成

class_mapping = { val:idx for idx, val in enumerate(np.unique(df.classlabel))}
class_mapping

# マッピング

_df = df.copy()
_df["classlabel"] = _df["classlabel"].map(class_mapping)
_df

,color,size,price,classlabel
0,green,M,10.1,0
1,red,L,13.5,1
2,blue,XL,15.3,0


In [35]:
#　マッピングを元に戻すには 反転した ディクショナリを使えば良い

inv_class_mapping = {v:k for k,v in class_mapping.items()}
inv_class_mapping

__df = _df.copy()
__df["classlabel"]  = __df["classlabel"].map(inv_class_mapping)
__df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [36]:
# sklean LabelEncoder

In [37]:
from sklearn.preprocessing import LabelEncoder


# ラベルエンコーダーのインスタンス生成
class_le = LabelEncoder()

# 整数に変換
y = class_le.fit_transform(df.classlabel)
y

array([0, 1, 0])

In [38]:
# 文字列に戻す

class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

In [50]:
_df = df.copy()
_df["size"] = _df["size"].map(size_mapping)
_df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [52]:
X = _df[["color", "size", "price"]].values
print(X)

print("====")
color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])

print(X)

[['green' 1 10.1]
 ['red' 2 13.5]
 ['blue' 3 15.3]]
====
[[1 1 10.1]
 [2 2 13.5]
 [0 3 15.3]]


In [63]:
# one-hot エンコーディング
from sklearn.preprocessing import OneHotEncoder

# インスタンス生成
ohe = OneHotEncoder(categorical_features=[0])

ohe.fit_transform(X).toarray()

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

In [65]:
# インスタンス生成
ohe = OneHotEncoder(categorical_features=[0], sparse=False)  # toarrayを省略
ohe.fit_transform(X)

array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

In [69]:
#pandas の場合はget_dummiesで

pd.get_dummies(
    df[["color", "size", "price"]]
)

,price,color_blue,color_green,color_red,size_L,size_M,size_XL
0,10.1,0,1,0,0,1,0
1,13.5,0,0,1,1,0,0
2,15.3,1,0,0,0,0,1
